<h2> More on Tensorflow </h2> 

GPUS dramatically speed up computations by splitting computations into many smaller chunks and running them in parallel across many GPU threads. 

In [2]:
import tensorflow as tf

In [6]:
#creating a simple tensor
t = tf.constant([[1,2,3],[4,5,6]])
t.shape

TensorShape([2, 3])

In [7]:
#indexing
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [5, 6]])>

In [8]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]])>

In [10]:
tf.matmul(t,tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

<h3> Tensors and Numpy </h3> 

In [11]:
import numpy as np
a = np.array([2,4,5])

In [13]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 5])>

In [23]:
#casting from 32bit float to a 64 bit float to sum in TF
t2 = tf.constant(40.,dtype = tf.float64)
t2

<tf.Tensor: shape=(), dtype=float64, numpy=40.0>

In [24]:
tf.constant(2.0) + tf.cast(t2,tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

<h3> Variables in Tensorflow </h3> 

We cannot modify constant tensors, however we can modify tf.Variable. These are required as we need to tweak weights in a neural network. 

In [25]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

A tf.Variable acts like a constant tensor and allows to perform the same operations. But it also allows you to modify th variable using the assign() method.

In [27]:
a = tf.constant([[1,2,3],[4,5,6]])
a

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [30]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [31]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [35]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates = [100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

In [36]:
v.scatter_nd_add(indices=[[0,0],[1,2]],updates = [100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[200.,  42.,   6.],
       [  8.,  10., 400.]], dtype=float32)>

scatter methods allows you to specify the index position and add to it

<h3> Sparse Tensors </h3> 

In a Sparse Tensor, you always need to give the positions first, then the values, then the dense shape

In [38]:
s = tf.SparseTensor(indices = [[0,1],[1,0],[2,3]],
                    values = [1.,2.,3.], 
                    dense_shape=[3,4])

In [40]:
tf.sparse.to_dense(s)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

Sparse tensors need to always be given in order of indices

In [47]:
s5 = tf.SparseTensor(indices=[[0, 1], [0, 2]],
                     values=[1., 2.],
                     dense_shape=[3, 4])

In [48]:
tf.sparse.to_dense(s5)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 2., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

<h3> Tensor Arrays </h3>

We first crrate the tensor with a fixed size. we cannot add past this

In [51]:
array = tf.TensorArray(dtype = tf.float32,size = 3)
array = array.write(0,tf.constant([1.,2.]))
array = array.write(1,tf.constant([3.,10.]))
array = array.write(2,tf.constant([5.,7.]))

<h3> Customizing models and training algorithms </h3> 

<h3> Custom Loss Functions </h3> 

In [56]:
#where y_true - y_pred absolute is less than 1, we want the squared loss and linear loss
def huber_fn(y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error,squared_loss,linear_loss)

#we need to always use tensorflow functions to be used as custom loss functions. 

For the next step, we just apply this to our keras model

model.compile(loss = huber_fn, optimizer = "adam")

model.fit(.....)

In [54]:
#current implementation allows us to have one threshold of 1 in the Huber Function
#How to change the threshold?
def create_huber(threshold = 1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_errors = tf.abs(error) <  threshold
        #mae loss
        squared_loss = tf.square(error)/2
        #mse loss
        linear_loss = threshold*tf.abs(error) - threshold**2/2
        #if your error is less than threshold, return squared loss, 
        #else return linear_loss
        return tf.where(is_small_errors, squared_loss,linear_loss)
    return huber_fn

model.compile(loss = create_huber(2.0), optimizer = "adam")

The above causes an issue when saving the model using 
keras.callbacks.ModelCheckpoint("modelname.h5", save_best_only = True)

When we load the model again we will have to specify which function is being called for the loss function as the loss function is a custom one here

model = keras.models.load_model("modelname.h5", custom_objects = {"huber_fn":create_huber(2.0)})

In [55]:
from tensorflow import keras

In [58]:
#to avoid the above we can inherit the Keras.losses.loss method

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold = 1.0, **kwargs):
        self.threshold = threshold
        #instantiate the superclass
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        #you get the base configuration of superclass
        base_config = super().get_config()
        #add in the threshold and creates a new dictionary to be returned
        return {**base_config, "threshold":self.threshold}

get_config method returns a dictionary mapping each hyperparameter name to its value. It first calls the parent class's get_config() method, then adds the new hyperparameters to this dictionary.

from the above, when loading a model, we do this:

**model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
custom_objects={"HuberLoss": HuberLoss})**

Now we dont have to provide the threshold value too.

When we save the mode, Keras calls the loss instance (HuberLoss) get_config() method and the returned dictionary is stored as a JSON in the h5 file. 

When loaded, it calls **from_config()** and creates the instance of the class, passing the return from **from_config()** to **kwargs.